In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import random
from heapq import heappop, heappush

from MAPF import Map
from MAPF import read_map_from_movingai_file, read_tasks_from_movingai_file

from CT import HighNode, LowNode

from OpenClosed import OpenHigh, OpenLow, ClosedLow

from Heuristics import ManhattanDistance

from CT import MakePath
from AstarTimeSteps import AstarTimesteps


In [2]:
# CBS with disjoint splitting, ничего не поменялось, только там где добавляются препятствия.

def CBSDS(gridMap, Starts, Goals, vertexCons, edgeCons): # vertex/edge- Cons - нужны только при вызове CBS в качестве нижнего уровня MACBS
    root = HighNode()
    OPEN = OpenHigh()
    CLOSED = ClosedHigh()
    agents = list(range(len(Starts)))
    k = 0
    
    for a in agents:
        planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], [], [])
        result = planner.FindPath()
        if result[0]:
            path = MakePath(result[1])
            root.sol[a], _ = path
        else:
            return (False, None, OPEN, CLOSED)
    
    root.g = sum([len(path) for path in root.sol.values()])
    OPEN.AddNode(root)
    k += 1
    
    while not OPEN.isEmpty():
        s = OPEN.GetBestNode(CLOSED)
        CLOSED.AddNode(s)  
        newVertexCons = []
        newEdgeCons = []
        
        for a in agents:
            for b in agents[a + 1 :]:
                for step in range(min(len(s.sol[a]), len(s.sol[b]))):
                    if s.sol[a][step].i == s.sol[b][step].i and s.sol[a][step].j == s.sol[b][step].j:
                        newVertexCons.append((a, b, (s.sol[a][step].i, s.sol[a][step].j), step))
                    if step + 1 < min(len(s.sol[a]), len(s.sol[b])) and \
                       s.sol[a][step].i == s.sol[b][step + 1].i and s.sol[a][step].j == s.sol[b][step + 1].j and \
                       s.sol[a][step + 1].i == s.sol[b][step].i and s.sol[a][step + 1].i == s.sol[b][step].i:
                        newEdgeCons.append((
                            a,
                            b,
                            (s.sol[a][step].i, s.sol[a][step].j),
                            (s.sol[a][step + 1].i, s.sol[a][step + 1].j),
                            step,
                        ))
                        
        if len(newVertexCons) == 0 and len(newEdgeCons) == 0:
            return (True, s, OPEN, CLOSED)
        
        # Сейчас сначала разрешаются вершинные конфликты, потом реберные
        if len(newVertexCons) > 0:
            a, b, (i, j), t = newVertexCons[0]
            
            # Разбиваем CT на ноды A и B, разрешая вершинный конфликт
            # первый нод останется таким же: запрещаем агенту а быть там где указано
            
            tmp = s.vertexCons.copy()
            if a in tmp:
                tmp[a].append(((i, j), t))   
            else:
                tmp[a] = [((i, j), t)]
            
            A = HighNode(
                vertexCons=tmp,
                edgeCons=s.edgeCons,
                sol=s.sol,
                parent=s,
                k=k,
            )
            
            ec = []
            if a in A.edgeCons:
                ec = A.edgeCons[a]
            
            planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], A.vertexCons[a], ec)
            result = planner.FindPath()
            if result[0]:
                path = MakePath(result[1])
                A.sol[a], _ = path
                A.g = sum([len(path) for path in A.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if not CLOSED.WasExpanded(A):
                    OPEN.AddNode(A)
                k += 1
                
            # Cо вторым агентом поступаем не так: не добавляем для него бебру, а наоборот обязываем агента 
            # а быть там где надо. 
            tmp = s.vertexCons.copy()
            if a in tmp:
                #запретить все кроме (i,j) в момент времени t
                for (x,y) in gridMap.cells():
                    if (x,y) != (i,j):
                        tmp[a].append(((x, y), t))   
            else:
                tmp[a] = []
                for (x,y) in gridMap.cells():
                    if (x,y) != (i,j):
                        tmp[a].append(((x, y), t)) 
                
            B = HighNode(
                vertexCons=tmp,
                edgeCons=s.edgeCons,
                sol=s.sol,
                parent=s,
                k=k,
            )
            #??? вопрос --- что в таком случае добавлять в эти ec? Видимо все так же для а...
            ec = []
            if a in B.edgeCons:
                ec = B.edgeCons[a]
            
            planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], B.vertexCons[a], ec)
            result = planner.FindPath()
            if result[0]:
                path = MakePath(result[1])
                B.sol[a], _ = path
                B.g = sum([len(path) for path in B.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if not CLOSED.WasExpanded(B):
                    OPEN.AddNode(B)
                k += 1
                
        elif len(newEdgeCons) > 0:
            a, b, (i1, j1), (i2, j2), t = newEdgeCons[0]
            
            # Разбиваем CT на ноды A и B, разрешая реберный конфликт 
            
            tmp = s.edgeCons.copy()
            if a in tmp:
                tmp[a].append(((i1, j1), (i2, j2), t))   
            else:
                tmp[a] = [((i1, j1), (i2, j2), t)]
            
            A = HighNode(
                vertexCons=s.vertexCons,
                edgeCons=tmp,
                sol=s.sol,
                parent=s,
                k=k,
            )
            
            vc = []
            if a in A.vertexCons:
                vc = A.vertexCons[a]
            
            planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], vc, A.edgeCons[a])
            result = planner.FindPath()
            if result[0]:
                path = MakePath(result[1])
                A.sol[a], _ = path
                A.g = sum([len(path) for path in A.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if not CLOSED.WasExpanded(A):
                    OPEN.AddNode(A)
                k += 1
                
            # обязываем а пройти по данному ребру в момент времени t. Это же так работает?(:    
            tmp = s.edgeCons.copy()
            if a in tmp:
                for (x,y) in gridMap.cells():
                    if (x,y) != (i1,j1) and (x+1,y) != (i2,j2):
                        tmp[a].append(((x, y), (x+1, y), t))   
                    elif (x,y) != (i1,j1) and (x-1,y) != (i2,j2):
                        tmp[a].append(((x, y), (x-1, y), t))  
                    elif (x,y) != (i1,j1) and (x,y-1) != (i2,j2):
                        tmp[a].append(((x, y), (x, y-1), t))  
                    elif (x,y) != (i1,j1) and (x,y+1) != (i2,j2):
                        tmp[a].append(((x, y), (x, y+1), t)) 
            else:
                tmp[a] = []
                for (x,y) in gridMap.cells():
                    if (x,y) != (i1,j1) and (x+1,y) != (i2,j2):
                        tmp[a].append(((x, y), (x+1, y), t))   
                    elif (x,y) != (i1,j1) and (x-1,y) != (i2,j2):
                        tmp[a].append(((x, y), (x-1, y), t))  
                    elif (x,y) != (i1,j1) and (x,y-1) != (i2,j2):
                        tmp[a].append(((x, y), (x, y-1), t))  
                    elif (x,y) != (i1,j1) and (x,y+1) != (i2,j2):
                        tmp[a].append(((x, y), (x, y+1), t)) 
            
            B = HighNode(
                vertexCons=s.vertexCons,
                edgeCons=tmp,
                sol=s.sol,
                parent=s,
                k=k,
            )
            
            vc = []
            if a in B.vertexCons:
                vc = B.vertexCons[a]
            
            planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], vc, B.edgeCons[a])
            result = planner.FindPath()
            if result[0]:
                path = MakePath(result[1])
                B.sol[a], _ = path
                B.g = sum([len(path) for path in B.sol.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if not CLOSED.WasExpanded(B):
                    OPEN.AddNode(B)
                k += 1
                
    return (False, None, OPEN, CLOSED)

In [3]:
map_file = 'tests/room-32-32-4.map'
scen_file = 'tests/scen-even/room-32-32-4-even-19.scen'


w, h, cells = read_map_from_movingai_file(map_file)
tasks = read_tasks_from_movingai_file(scen_file)

task_map = Map()
task_map.SetGridCells(w, h, cells)

num = 4

Starts = [(item[0], item[1]) for item in tasks[2:num+2]]
Goals = [(item[2], item[3]) for item in tasks[2:num+2]]

result = CBSDS(task_map, Starts, Goals)

if result[0]:
    print(f'Final cost {result[1].g},\nEuclidean cost {sum([tasks[i][-1] for i in range(num)])}')
    
    obs_x = []
    obs_y = []

    for y, cell_y in enumerate(task_map.cells):
        for x, cell_x in enumerate(cell_y):
            if cell_x == 1:
                obs_x.append(x)
                obs_y.append(y)
            
    x = np.array(obs_x)
    y = np.array(obs_y)

    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)
    ax.set_ylim(task_map.height, 0)
    ax.xaxis.set_ticks_position('top')

    ax.scatter(x, y, c='black', marker='s', s=50)
    ax.set_facecolor('white')
    ax.set_title(map_file)

    color = ["#" + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) + ''.join(['A', '0'])
             for i in range(num)]

    for a in result[1].sol:
        path = result[1].sol[a]
        x = []
        y = []
        for node in path:
            x.append(node.i)
            y.append(node.j)
        ax.scatter(x[1:-1], y[1:-1], c=color[a], marker='s', s=50)
        ax.scatter(x[:1], y[:1], c='blue', marker='s', s=50)
        ax.scatter(x[-1:], y[-1:], c='red', marker='s', s=50)

    plt.show()

TypeError: CBSDS() missing 2 required positional arguments: 'vertexCons' and 'edgeCons'